In [39]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math

In [40]:
# Read dataframes
df_Market_Cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_Employees = pd.read_pickle("UsableData/Employees.pkl")
df_Book_Value = pd.read_pickle("UsableData/BookValue.pkl")
df_Variance = pd.read_pickle("UsableData/Variance.pkl")
df_Returns = pd.read_pickle("UsableData/Returns.pkl").T

In [41]:
# Use existing dataframe as template and set all cloumns to False and data type to Boolean
df_eligible = df_Market_Cap.copy(deep=True)
for column in df_eligible:
    df_eligible[column].astype("bool")
    df_eligible[column] = False

In [42]:
# Create method that checks if there are non NaN returns in one year and therefore the stock is usable in that year in regards to the return data
def unusableReturns(ric, year):
    selection = df_Returns[ric][(df_Returns.index >= f"{year}-01-01") & (df_Returns.index <= f"{year}-12-31")]
    return bool(selection.isna().values.any())

In [43]:
# Iterate through every year and see which stocks are eligible
years = df_Market_Cap.columns.to_numpy()[:-1] # Exclude first year, because no data from previous year to calculate weights
for ric in df_Market_Cap.index: # Go through every stock
    for year in years: # In every year
        mc = df_Market_Cap.loc[ric, year]
        emp = df_Employees.loc[ric, year]
        bv = df_Book_Value.loc[ric, year]
        var = df_Variance.loc[ric, year]
        if(math.isnan(mc) or math.isnan(emp) or math.isnan(bv) or math.isnan(var) or unusableReturns(ric, year+1)): # If one of the data for weights (mc, emp, bv, var) is nan or return for next year stock is not eligible
            df_eligible.at[ric, year+1] = False
        else: # If all the data is there, stock is eligible
            df_eligible.at[ric, year+1] = True
df_eligible.drop(columns=[years[0]], inplace=True) # Drop first and last year
df_eligible.columns

Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021],
      dtype='int32', name='Year')

In [44]:
# Count how many stocks are eligible in each year
for i in range(len(df_eligible.columns)):
    df = df_eligible[df_eligible[df_eligible.columns[i]] == True]
    print(f"{df_eligible.columns[i]}: {len(df.index)}")

1996: 2
1997: 22
1998: 33
1999: 104
2000: 194
2001: 284
2002: 344
2003: 370
2004: 432
2005: 447
2006: 490
2007: 554
2008: 578
2009: 588
2010: 555
2011: 571
2012: 582
2013: 552
2014: 534
2015: 471
2016: 450
2017: 442
2018: 449
2019: 456
2020: 487
2021: 525


In [45]:
# Write dataframe
df_eligible.to_pickle("UsableData/EligibleStocks.pkl")